<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="line-height: 1.3">E6156: Topics in SW Engineering - Microservices and Cloud Applications.<br>DynamoDB, Lambda Functions, API Gateway, Web APIs, DynamoDB

## Lecture Overview



## Next Tasks

- Our next tasks will be to
    - Add support for managing a user's profile.
    - Email ownership validation.
    - Facebook logon and OAuth2
    - Get everything behind CloudFront and the API Gateway.
    - Lambda functions
    

## User Profile

### Overview

| <img src="./images/profile.jpeg" > |
| :---: |
| __User Profile__ |

- The user profile has some basic information.


- There is also a set of loosely typed entries. Programmers can add their own
    - JSON-like types to the profile "schema"
    - Add application elements that understand existing a new types.
    
    
- Older code just ignores the types it does not understand.


- This loose-typing is almost impossible to implement in the relational model. We do not want to manage any kind of server, so we will implement the profile service using
    - AWS Lambda functions.
    - DynamoDB
    

## Our Data Model

- The basic format of a profile and an example is:

```
{
  "created_timestamp": 1539264470716,
  "customer_id": "dofe1",
  "modified_timestamp": 1539264470716,
  "profile_entries": [
    {
      "label": "office_address",
      "value": {
        "city": "New York",
        "country": "USA",
        "post_code": "10027-6601",
        "state": "NY",
        "street1": "520 W 120th St",
        "street2": null
      }
    },
    {
      "label": "admin_contact_info",
      "value": {
        "email": "sith_apprentice@deathstar.gov",
        "Note": "Try to void aggravation",
        "phone": "212-555-1212"
      }
    }
  ]
}
```

- Application and sets of cooperating applications use the profile to save/retrieve useful information.


- A ```ProfileService``` manages:
    - ```Profiles```
    - ```ProfileEntryTypes```
    
    
- A ```ProfileEntryType``` has the format:

```
{
  "profile_label": "admin_contact_info",
  "owner": "donfe1",
  "description": "Obvious explanation",
  "Example": {
    "contact_name": "Darth Vader",
    "contact_email": "dv@deathstar.gov",
    "comments": "You get used to the hissing sound after a while."
  }
}
```

- The example is useful for designing and populating web forms, enabling programmers, etc.

### DynamoDB Concept

| <img src="./images/dynamodb_partitions.png"> |
| :---: |
| [DynamoDB Concept](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/HowItWorks.Partitions.html)

- A DynamoDB "table" has one or more _partitions._ and must define a _partition key_ that is a data field on ALL entries in the table.


- DynamoDB processes operations by:
    - Getting the partition key value from the request data.
    - Hashing to determine the partition.
    - Routing the request to the partition that contains (may contain) the data.
    

### Some Database Server/Data Layer Design Patterns

- There are two [basic approaches to scalability:](https://en.wikipedia.org/wiki/Scalability#Horizontal_and_vertical_scaling)
    - Horizontal aka scale-out: add more (or less) systems.
    - Vertical aka scale-up: Use a bigger system (more CPUs, memory, disk, ...)
    
| <img src="./images/scale_up_out.jpeg"> |
| :---: |
| __Scale-Up and Scale-Out__ |

 - There are pros and cons to each approach.
 
 
 - RDB's basic pattern is scale-up. DynamoDB's pattern is scale-out.
 
 
 - You can also think in terms of _shared everything_ versus _shared nothing._
 
| <img src="./images/shared-nothing-comparison.jpg" width="900px"> |
| :---: |
| [Shared Everything vs. Shared Nothing](https://www.morpheusdata.com/blog/shared-nothing-architecture) |

- RDBs tend to favor _shared everything._ DynamoDB is _shared nothing._


- Shared nothing/scale out works
    - Extremely well for:
        - Scans
        - Individual item read/write/update.
    - Poorly for:
        - Referential integrity.
        - Multi-table/type operations, e.g. JOIN.
        
- Shared everything/scale up works
    - Extremely well for:
        - Complex, multi-table/type queries.
        - Referential integrity.
    - Poorly for:
        - Massive scalability.
        - Availability.
        
        
- I conflated scale up/versus scale out with shared everything/shared nothing. There is some ability to mix and match, but in general the two patterns are scale out/shared nothing and scale up/share everything.

### The "CAP" Theorem

"In theoretical computer science, the CAP theorem, also named Brewer's theorem after computer scientist Eric Brewer, states that it is impossible for a distributed data store to simultaneously provide more than two out of the following three guarantees:
- _Consistency:_ Every read receives the most recent write or an error.
- _Availability:_ Every request receives a response that is not an error.
- _Partition tolerance:_ The system continues to operate despite an arbitrary number of messages being dropped (or delayed) by the network between nodes.
    
In particular, the CAP theorem implies that in the presence of a network partition, one has to choose between consistency and availability. Note that consistency as defined in the CAP theorem is quite different from the consistency guaranteed in ACID database transactions." (https://en.wikipedia.org/wiki/CAP_theorem)

| <img src="./images/cap_theorem.png" width="900px"> |
| :---: |
| [CAP Theorem and DB Engines/Models] (https://blog.flux7.com/blogs/nosql/cap-theorem-why-does-it-matter) |

- Why the _consistency_ versus _availability_ trade-off?
    - If there is a single copy of each data item,  the system can ensure consistency for the item but not availability.
    - Replicated copies ensures availability, but _consistency_ would require some form of _locking_ of the replicas to perform a write. This is impossible if there are network partitions.

### NoSQL Databases

"A NoSQL (originally referring to "non SQL" or "non relational") database provides a mechanism for storage and retrieval of data that is modeled in means other than the tabular relations used in relational databases. Such databases have existed since the late 1960s, but did not obtain the "NoSQL" moniker until a surge of popularity in the early twenty-first century, triggered by the needs of Web 2.0 companies. NoSQL databases are increasingly used in big data and real-time web applications. NoSQL systems are also sometimes called "Not only SQL" to emphasize that they may support SQL-like query languages, or sit alongside SQL database in a polyglot persistence architecture.

Motivations for this approach include: simplicity of design, simpler "horizontal" scaling to clusters of machines (which is a problem for relational databases), and finer control over availability. The data structures used by NoSQL databases (e.g. key-value, wide column, graph, or document) are different from those used by default in relational databases, making some operations faster in NoSQL. The particular suitability of a given NoSQL database depends on the problem it must solve. Sometimes the data structures used by NoSQL databases are also viewed as "more flexible" than relational database tables" (https://en.wikipedia.org/wiki/NoSQL)


There are two basic reasons two use a "Not Only SQL Database:"
- The application does not require RDB functions, which means a reduced function DB can be cheaper, faster, more available, ...
- The application requires capabilities that are hard to achieve in the relational model.

### Our Use of DynamoDB

- Our application will not scale to the point where we need DynamoDB.


- We are using DynamoDB
    - To get a feel for NoSQL models.
    - Understand DynamoDB and similar programming models.
    - Implementing our profile data model would be "icky" on relational models.

### DynamoDB Python API

#### Functions

|                 |                     |
|----------------------------------|----------------------------------|
| batch_get_item()                 | get_waiter()                     |
| batch_write_item()               | list_backups()                   |
| can_paginate()                   | list_global_tables()             |
| create_backup()                  | list_tables()                    |
| create_global_table()            | list_tags_of_resource()          |
| create_table()                   | put_item()                       |
| delete_backup()                  | query()                          |
| delete_item()                    | restore_table_from_backup()      |
| delete_table()                   | restore_table_to_point_in_time() |
| describe_backup()                | scan()                           |
| describe_continuous_backups()    | tag_resource()                   |
| describe_endpoints()             | untag_resource()                 |
| describe_global_table()          | update_continuous_backups()      |
| describe_global_table_settings() | update_global_table()            |
| describe_limits()                | update_global_table_settings()   |
| describe_table()                 | update_item()                    |
| describe_time_to_live()          | update_table()                   |
| generate_presigned_url()         | update_time_to_live()            |
| get_item()                       |   get_paginator()                |
| | [DynamoDB Python API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html) |

- Lot's of obvious operations for reading, writing, getting metadata, etc.


- Unlike a DB, you must use pagination. There are no cursors or fetchall().


- No functions like JOIN. 


- The primary functions are:
    - ```get_item(), delete_item()``` and ```put_item(),``` which are link select, delete and insert using a primary key for an RDB.
    - ```scan(),``` which is like a ```select ... where ... ``` but using a different format for project and select clauses.
    - There is also support for batch operations, because the primary focus is web scale performance.

### Examples

#### Overview

- I downloaded and am using the "local" DynamoDB for these tests. See https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/DynamoDBLocal.html.


- I used the AWS Command Line (CLI) to create the local tables.

```
aws dynamodb create-table --table-name
CustomerProfileTypes --attribute-definitions AttributeName=profile_label,AttributeType=S
--key-schema AttributeName=profile_label,KeyType=HASH
--provisioned-throughput ReadCapacityUnits=5,WriteCapacityUnits=5 --endpoint-url http://localhost:8000
```


#### Setup the Client

In [1]:
import boto3

client = boto3.client('dynamodb')
from boto3.dynamodb.conditions import Key, Attr

# There is some weird stuff in DynamoDB JSON responses. These utils work better.
from dynamodb_json import json_util as jsond

dynamodb = boto3.resource('dynamodb', endpoint_url='http://localhost:8000')

#### Get and Item from a Table

In [2]:
def do_a_get(table_name, key):
    table = dynamodb.Table(table_name)

    response = table.get_item(
        Key=key
    )

    print("GetItem succeeded:")
    return response
    #print(json.dumps(response, indent=4))

In [3]:
response = do_a_get('CustomerProfileTypes', key = {
        "profile_label" : "office_contact_info"
    })
print(json.dumps(response, indent=4))

EndpointConnectionError: Could not connect to the endpoint URL: "http://localhost:8000/"

#### Get Table Metadata

In [52]:
def describe_table(table_name):
    client = boto3.client('dynamodb', endpoint_url='http://localhost:8000')
    result = client.describe_table(TableName='CustomerProfileTypes')
    print(jsond.dumps(result, indent=3))

In [14]:
describe_table('CustomerProfileTypes')

{
   "Table": {
      "M": {
         "AttributeDefinitions": {
            "L": [
               {
                  "M": {
                     "AttributeName": {
                        "S": "profile_label"
                     },
                     "AttributeType": {
                        "S": "S"
                     }
                  }
               }
            ]
         },
         "TableName": {
            "S": "CustomerProfileTypes"
         },
         "KeySchema": {
            "L": [
               {
                  "M": {
                     "AttributeName": {
                        "S": "profile_label"
                     },
                     "KeyType": {
                        "S": "HASH"
                     }
                  }
               }
            ]
         },
         "TableStatus": {
            "S": "ACTIVE"
         },
         "CreationDateTime": {
            "S": "2018-10-11T14:22:42.917000"
         },
         "ProvisionedThrough

#### Put an Item

In [54]:
def do_a_put(table_name, item):
    print("Hello")
    table = dynamodb.Table(table_name)
    Item={
        'year': 1,
        'title': 1,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': 0
        }
    }

    response = table.put_item(
        Item=item
    )
    #print("Put response = ", json.dumps(response))
    return response

In [55]:
result = do_a_put('CustomerProfileTypes',
        {
            "description": "Obvious explanation",
            "Example": {
                "comments": "Push the really cool red button! Flip a lever. Toggle some switches.",
                "training_url" : "https://www.youtube.com/watch?v=djZFHTa6TfA"
             },
            "owner": "donfe1",
            "profile_label": "neat_weapons"
        })
print(json.dumps(result, indent=4))

Hello
{
    "ResponseMetadata": {
        "RequestId": "d04e3ac4-4988-435e-accb-5815e4868ab2",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "x-amz-crc32": "2745614147",
            "x-amzn-requestid": "d04e3ac4-4988-435e-accb-5815e4868ab2",
            "content-length": "2",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0
    }
}


    
- The first thing we will support in the profile is various addresses.


- People are really, really bad at entering addresses.


- We want to simplify the task of entering a correct, valid address.


- We will use SmartyStreets.com. There are two parts:
    - A UI plug-in
    - A validation API.

#### Scan a Table

In [56]:
def do_a_scan(table_name, filterexpression):
    table = dynamodb.Table(table_name)

    response = table.scan(
        filterexpression
    )

    print("Scan succeeded")
    #print(json.dumps(response, indent=4))
    return response

In [44]:
FilterExpression = Attr('owner').eq('donfe1')
response = do_a_scan('CustomerProfileTypes', FilterExpression)
print("Scan = ", json.dumps(response, indent=2))


Scan succeeded
Scan =  {
  "Items": [
    {
      "owner": "donfe1",
      "description": "Obvious explanation",
      "profile_label": "admin_contact_info",
      "Example": {
        "contact_name": "Darth Vader",
        "comments": "You get used to the hissing sound after a while.",
        "contact_email": "dv@deathstar.gov"
      }
    },
    {
      "owner": "donfe1",
      "description": "Obvious explanation",
      "profile_label": "neat_weapons",
      "Example": {
        "comments": "Push the really cool red button! Flip a lever. Toggle some switches.",
        "training_url": "https://www.youtube.com/watch?v=djZFHTa6TfA"
      }
    },
    {
      "owner": "donfe1",
      "description": "Obvious explanation",
      "profile_label": "office_contact_info",
      "Example": {
        "contact_name": "Darth Vader",
        "comments": "You get used to the hissing sound after a while.",
        "contact_email": "dv@deathstar.gov"
      }
    }
  ],
  "Count": 3,
  "ScannedCount

### An Observation $-$ The DAOs Again

| <img src="./images/dao_again.jpeg" > |
| :---: |
| __Not this again!__ |


- Fooling around, I called the functions ```do_a_get(), do_a_put()``` and ```do_a_scan().```


- The functions required some basic connection and configuration information.


- If,
    - I had called the functions ```create(), retrieve_by_template()``` and ```retrieve_by_primary_key()```
    - And put the functions in a class.
    - And used JavaScript
    
    
- I would have had a DAO remarkably similar to the RelationalDAO.


- This was the point I was trying to make around DAOs. For simple functions,
    - You can write common application code.
    - That is isolated from details of the actual DB API, implementation, data formats, verbs, etc.
    
    
- Which enables flexibility, agility, ...


- Of course, you often have to go around the DAO for more complex things, but ...
    - Simple things should be simple.
    - Two solutions/APIs to the same problem (data access) should be different if and only if there is a compelling reason.

## Lambda Functions

### Our Solution (Status)

| <img src="./images/project_comp_status_2.jpg"> |
| :---: |
| __Project Components__ |


- We will use Lambda functions for
    - Profile information.
    - Address verification.
    - Email (and phone) ownership verification.

### Concept

"Serverless computing is a cloud-computing execution model in which the cloud provider acts as the server, dynamically managing the allocation of machine resources. Pricing is based on the actual amount of resources consumed by an application, rather than on pre-purchased units of capacity. It is a form of utility computing.

The name "serverless computing" is used because the server management and capacity planning decisions are completely hidden from the developer or operator. Serverless code can be used in conjunction with code deployed in traditional styles, such as microservices. Alternatively, applications can be written to be purely serverless and use no provisioned servers at all." (https://en.wikipedia.org/wiki/Serverless_computing)


"Function as a service (FaaS)" is a category of cloud computing services that provides a platform allowing customers to develop, run, and manage application functionalities without the complexity of building and maintaining the infrastructure typically associated with developing and launching an app. Building an application following this model is one way of achieving a "serverless" architecture, and is typically used when building microservices applications."


- How does "serverless" compare to PaaS, e.g Elastic Beanstalk
    - The zip file more or less contained an application "server."
    - You are aware of stopping, starting, etc. the application server.
    - There are considerations around scaling levels, etc.
    
    
- Serverless is:
    - Event happens.
    - Bound to a function (Lambda function).
    - Environment starts, executes function and terminates.

| <img src="./images/lambda_concept.jpg"> |
| :---: |
| __Project Components__ |

### API Gateway

"API management is the process of creating and publishing web APIs, enforcing their usage policies, controlling access, nurturing the subscriber community, collecting and analyzing usage statistics, and reporting on performance." (https://en.wikipedia.org/wiki/API_management)

- Making money on the web is often about APIs, not UIs.


- AWS API Gateway is the core of an API management solution. Conceptually provides a simple place for API
    - Security.
    - Monitoring.
    - Change management.
    - Throttling, metering, resource management.
    - Logging.
    - Analytics.
    - etc.

### Simplest Combination: API Gateway and Lambda Function



#### Lambda Function

In [57]:
import boto3
import json

print('Loading function')
dynamo = boto3.client('dynamodb')


def respond(err, res=None):
    print("err = ", err)
    print("rsp = ", res)
    rsp = {
        'statusCode': '400' if err else '200',
        'body': err.message if err else json.dumps(res),
        'headers': {
            'Content-Type': 'application/json',
        },
    }
    print("Response = ", rsp)
    return rsp


def lambda_handler(event, context):
    '''Demonstrates a simple HTTP endpoint using API Gateway. You have full
    access to the request and response payload, including headers and
    status code.'''

    print("Received event: " + json.dumps(event, indent=2))

    response = None
    error = None

    if event['httpMethod'] == 'GET':
        response = {"msg": "You called GET."}
    elif event['httpMethod'] == 'POST':
        response = {"msg": "You called POST."}
    elif event['httpMethod'] == 'PUT':
        response = {"msg": "You called PUT."}
    elif event['httpMethod'] == 'DELETE':
        response = {"msg": "You called DELETE."}
    else:
        error = {"msg": "I cannot deal with this today. Seriously."}

    if response:
        path_params = event.get('pathParameters', None)
        response['path_params'] = path_params
        q_params = event.get('queryStringParameters', None)
        response['query_params'] = q_params
        body = event.get('body', None)
        response['body'] = body
        auth = event.get('authentication', None)
        response['authentication'] = auth

    return respond(error, response)


def do_dynamo_stuff(event, context):
    # Get the service resource.
    dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('CustomerProfileTypes')
    try:
        response = table.get_item(
            Key={
                'profile_label': 'admin_contact_info'
            })
        print("Response = ", json.dumps(response['Item'], indent=2))
        return respond(None, response['Item'])
    except Exception as e:
        print("Exception e = ", e)

Loading function


#### Local test

In [58]:
event = {
  "body": "eyJ0ZXN0IjoiYm9keSJ9",
  "resource": "/{proxy+}",
  "path": "/path/to/resource",
  "httpMethod": "POST",
  "isBase64Encoded": "false",
  "queryStringParameters": {
    "foo": "bar"
  },
  "pathParameters": {
    "proxy": "/path/to/resource"
  },
  "stageVariables": {
    "baz": "qux"
  },
  "headers": {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, sdch",
    "Accept-Language": "en-US,en;q=0.8",
    "Cache-Control": "max-age=0",
    "CloudFront-Forwarded-Proto": "https",
    "CloudFront-Is-Desktop-Viewer": "true",
    "CloudFront-Is-Mobile-Viewer": "false",
    "CloudFront-Is-SmartTV-Viewer": "false",
    "CloudFront-Is-Tablet-Viewer": "false",
    "CloudFront-Viewer-Country": "US",
    "Host": "1234567890.execute-api.us-east-1.amazonaws.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Custom User Agent String",
    "Via": "1.1 08f323deadbeefa7af34d5feb414ce27.cloudfront.net (CloudFront)",
    "X-Amz-Cf-Id": "cDehVQoZnx43VYQb9j2-nvCh-9z396Uhbp027Y2JvkCPNLmGJHqlaA==",
    "X-Forwarded-For": "127.0.0.1, 127.0.0.2",
    "X-Forwarded-Port": "443",
    "X-Forwarded-Proto": "https"
  },
  "requestContext": {
    "accountId": "123456789012",
    "resourceId": "123456",
    "stage": "prod",
    "requestId": "c6af9ac6-7b61-11e6-9a41-93e8deadbeef",
    "requestTime": "09/Apr/2015:12:34:56 +0000",
    "requestTimeEpoch": 1428582896000,
    "identity": {
      "cognitoIdentityPoolId": "null",
      "accountId": "null",
      "cognitoIdentityId": "null",
      "caller": "null",
      "accessKey": "null",
      "sourceIp": "127.0.0.1",
      "cognitoAuthenticationType": "null",
      "cognitoAuthenticationProvider": "null",
      "userArn": "null",
      "userAgent": "Custom User Agent String",
      "user": "null"
    },
    "path": "/prod/path/to/resource",
    "resourcePath": "/{proxy+}",
    "httpMethod": "POST",
    "apiId": "1234567890",
    "protocol": "HTTP/1.1"
  }
}

In [59]:
def test1():
    result = lambda_handler(event, None)
    body = json.loads(result['body'])
    print("test1 result = ", json.dumps(result, indent=2))
    print("RSP body = ", json.dumps(body, indent=2))

test1()

Received event: {
  "body": "eyJ0ZXN0IjoiYm9keSJ9",
  "resource": "/{proxy+}",
  "path": "/path/to/resource",
  "httpMethod": "POST",
  "isBase64Encoded": "false",
  "queryStringParameters": {
    "foo": "bar"
  },
  "pathParameters": {
    "proxy": "/path/to/resource"
  },
  "stageVariables": {
    "baz": "qux"
  },
  "headers": {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, sdch",
    "Accept-Language": "en-US,en;q=0.8",
    "Cache-Control": "max-age=0",
    "CloudFront-Forwarded-Proto": "https",
    "CloudFront-Is-Desktop-Viewer": "true",
    "CloudFront-Is-Mobile-Viewer": "false",
    "CloudFront-Is-SmartTV-Viewer": "false",
    "CloudFront-Is-Tablet-Viewer": "false",
    "CloudFront-Viewer-Country": "US",
    "Host": "1234567890.execute-api.us-east-1.amazonaws.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Custom User Agent String",
    "Via": "1.1 08f323deadbeefa7af34d5feb414ce27

#### In the Cloud

| <img src="./images/api_gw.jpeg" > |
| :---: |
| __API Gateway Configuration__ |

| <img src="./images/lambda_cloud.jpeg" >|
| :---: |
| __Lambda Function Deployment__ |

- Demo and walkthroughs.

## Backup

### Step 1: Expediting Entering Valid Addresses

- This is not a "web commerce" class, but understanding basic concepts is useful.


- You want to minimize the number of "clicks to transaction." This is why Amazon patented "1-Click" checkout, and so many sites added the concept when the patent expired.


- If you ask me to enter an address and then reject as invalid N times, I am going to buy from someone who makes things easier.


- So, we will add automated, free form address completion to our UI.

| <img src="./images/address_freeform.jpeg"> |
| :---: |
| __Free Form Address Completion__ |


- HTML Snippet:

```
<div class="card text-white bg-secondary my-4">
        <div class="card-body">
            <h2>Enter and Address</h2>
            <form>
                <br>
                    <input type="text" id="newaddress" name="newaddress" placeholder="{{placeholder}}"  style="width:500px">
                <br>
            </form>
            <p></p>
            <div class="dropdown">
                <button class="btn  btn-primary dropdown-toggle"
                        type="button" id="dropdownMenuButton" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false" style="color:white;" >
                    Choose the type of address
                </button>
                <div class="dropdown-menu" aria-labelledby="dropdownMenuButton">
                    <a ng-click="addressKind($index)" ng-repeat="k in addressKinds" class="dropdown-item">{{k}}</a>
                </div>
            </div>
        </div>
    </div>
```

- Controller Snippet:

```
 let s3 = jQuery.LiveAddress({
        key: "xxxxxx384552786",
        waitForStreet: true,
        debug: true,
        target: "US",
        placeholder: "Enter address",
        addresses: [{
            freeform: '#newaddress'
        }]
    });

    s3.on("AddressAccepted", function(event, data, previousHandler)
    {
        console.log("Boo Yah!")

    });

    $scope.placeholder = "enter an address and select a choice."

    $scope.addressKinds = ['Home', 'Work', 'Other']

    $scope.addressKind = function(idx) {
        console.log("Address kink = " + $scope.addressKinds[idx]);
    };
```

- In ```s3.on("AddressAccepted", function() { ... ... })```, the controller will call the ```CustomerProfileService``` to save the profile entry.

- Demo of UI and SmartyStreets goes here.


- You team should sign-up for a free developer account.